# Final Project

Group Members:

1. Saanavi Goyal
2. Vanessa Campos
3. Jannat Nabonee

*Primary Objective:*  Use Berry Cox’s price momentum factors to select basket of assets for a long-short
portfolio. Backtest your algorithm with 5 years of historical data to check for performance of algorithm.

*Background:* The use of computers and coding allows finance professionals to identify new alpha factors –
Alpha factors are ways of quantifying where alpha is being generated by an asset. You will be responsible
for using Berry Cox’s list of alpha factors to determine the long/short baskets for your portfolio. But in
order to test your code and its asset selections, back-testing must be performed.

**Requirements**

Step 1: Choose an ETF with a minimum of 100 assets, identify those assets

Step 2: Retrieve historical data for your chosen ETF

Step 3: Calculate the price momentum factors for each asset in your ETF

Step 4: Using the price momentum factors, calculate the monthly z-factor score for each asset

Step 5: Identify long and short baskets (10 to 15 assets in each) using calculated z-factors

Step 6: Create a backtest to validate performance of your algorithm based on monthly restructuring over the
previous 5 years.

Step 7: Chart:
1. Monthly portfolio return bar chart (pos/neg coloring) vs ETF
2. Monthly return for long picks vs short picks vs ETF
3. Cumulative portfolio return vs ETF

In [ ]:
pip install yfinance

In [ ]:
pip install requests

In [ ]:
#Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf
import datetime as dt
from datetime import timedelta
import seaborn as sns
from bs4 import BeautifulSoup
import requests

**Step 1:** Choose an ETF with a minimum of 100 assets, identify those assets

ETF: SPY - S&P 500

In [ ]:
# Define the URL
url = "https://www.ssga.com/us/en/individual/etfs/library-content/products/fund-data/etfs/us/holdings-daily-us-en-spy.xlsx"

# Download the file
response = requests.get(url)
if response.status_code == 200:
    with open("holdings-daily-us-en-spy.xlsx", "wb") as file:
        file.write(response.content)
    print("File downloaded successfully!")
else:
    print(f"Failed to download file. HTTP Status Code: {response.status_code}")

# Load the Excel file. Skip 4 rows to ensure column names are read first
sp500_df = pd.read_excel("holdings-daily-us-en-spy.xlsx", skiprows=4)


# Extract tickers (SPY holdings) from the 'Ticker' column
spy_holdings = sp500_df['Ticker'].dropna().tolist()

# Replace ".B" with "-B" in ticker symbols, this is the listing for Berkshire Hathaway security
spy_holdings = sp500_df["Ticker"].dropna().replace(["BRK.B", "BF.B"],["BRK-B", "BF-B"])

# Remove "-" from the list, this is the listing for cash in the ETF and convert to list
spy_holdings = spy_holdings[spy_holdings != "-"].to_list()

# Display the first few tickers
print("Extracted Tickers:", spy_holdings[:10])


File downloaded successfully!
Extracted Tickers: ['AAPL', 'NVDA', 'MSFT', 'AMZN', 'META', 'GOOGL', 'TSLA', 'BRK-B', 'GOOG', 'AVGO']


[link text](https://)
**Step 2:** Retrieve historical data for your chosen ETF (10 Years)

In [ ]:
# Downloading data using SPY assets
#Credit to https://github.com/CCNY-Analytics-and-Quant/Quantitative_Finance_Spring2024_AS/blob/main/Sharma_Addhyaya_Final_Project.ipynb
start_date = dt.datetime.today() - dt.timedelta(days=252*10) # 10 years data
end_date = dt.datetime.today()
my_port = yf.download(start=start_date, end=end_date, tickers=spy_holdings)
my_port  # Visualizing stock data

[*********************100%***********************]  503 of 503 completed


Price                       Adj Close                                      \
Ticker                              A        AAPL        ABBV        ABNB   
Date                                                                        
2018-01-04 00:00:00+00:00   65.440277   40.705479   73.058868         NaN   
2018-01-05 00:00:00+00:00   66.486542   41.168930   74.330681         NaN   
2018-01-08 00:00:00+00:00   66.629242   41.016018   73.139755         NaN   
2018-01-09 00:00:00+00:00   68.265205   41.011318   73.691093         NaN   
2018-01-10 00:00:00+00:00   67.333099   41.001911   73.286766         NaN   
...                               ...         ...         ...         ...   
2024-11-21 00:00:00+00:00  132.059998  228.520004  171.729996  133.259995   
2024-11-22 00:00:00+00:00  133.839996  229.869995  176.949997  137.339996   
2024-11-25 00:00:00+00:00  134.490005  232.869995  177.059998  140.929993   
2024-11-26 00:00:00+00:00  134.369995  235.059998  181.139999  139.669998   
2024-11-27 00:00:00+00:00  138.139999  234.929993  183.080002  138.279999   

Price                                                                      \
Ticker                            ABT        ACGL         ACN        ADBE   
Date                                                                        
2018-01-04 00:00:00+00:00   52.074825   28.118069  140.948975  183.220001   
2018-01-05 00:00:00+00:00   52.225338   28.007132  142.111710  185.339996   
2018-01-08 00:00:00+00:00   52.074825   28.010302  143.247360  185.039993   
2018-01-09 00:00:00+00:00   52.163368   27.648960  143.725067  186.699997   
2018-01-10 00:00:00+00:00   52.083683   27.496817  143.220322  187.110001   
...                               ...         ...         ...         ...   
2024-11-21 00:00:00+00:00  117.260002   98.589996  361.049988  504.440002   
2024-11-22 00:00:00+00:00  117.760002   99.690002  358.660004  512.150024   
2024-11-25 00:00:00+00:00  118.129997  100.059998  361.290009  518.729980   
2024-11-26 00:00:00+00:00  117.959999  100.410004  363.179993  525.299988   
2024-11-27 00:00:00+00:00  118.949997  100.739998  362.160004  513.679993   

Price                                             ...   Volume           \
Ticker                            ADI        ADM  ...      WTW       WY   
Date                                              ...                     
2018-01-04 00:00:00+00:00   80.361885  33.174355  ...   800900  2727800   
2018-01-05 00:00:00+00:00   80.687569  32.952927  ...   646700  2924800   
2018-01-08 00:00:00+00:00   80.828400  32.879097  ...   632700  2760800   
2018-01-09 00:00:00+00:00   80.661163  32.985737  ...   560200  2530300   
2018-01-10 00:00:00+00:00   79.314453  32.731491  ...  1131200  4519700   
...                               ...        ...  ...      ...      ...   
2024-11-21 00:00:00+00:00  213.960007  53.189999  ...   615800  2763400   
2024-11-22 00:00:00+00:00  214.589996  53.130001  ...   525700  2514100   
2024-11-25 00:00:00+00:00  223.580002  53.009998  ...  1118600  7706000   
2024-11-26 00:00:00+00:00  219.050003  53.720001  ...   641600  2365400   
2024-11-27 00:00:00+00:00  217.169998  54.369999  ...  1204700  2785100   

Price                                                                    \
Ticker                        WYNN      XEL       XOM      XYL      YUM   
Date                                                                      
2018-01-04 00:00:00+00:00  2708800  2807000  10863000   796500  1971200   
2018-01-05 00:00:00+00:00  2186600  3728000  11047600   666800  1927100   
2018-01-08 00:00:00+00:00  1803800  3837600  10927100   554700  1599400   
2018-01-09 00:00:00+00:00  2453600  3006300   8131600   687600  1701000   
2018-01-10 00:00:00+00:00  2296800  2690000  10822300   784900   887100   
...                            ...      ...       ...      ...      ...   
2024-11-21 00:00:00+00:00  1612700  2644900  14675400  1686200  1031100   
2024-11-22 00:00:00+00:00  1620000  20653

**Step 3:** Calculate the price momentum factors for each asset in your ETF

In [ ]:
# Calculate daily returns
adj_close = my_port['Adj Close']
daily_port = adj_close.pct_change()

# Drop columns with all NaN values
daily_port.dropna(axis=1, how='all', inplace=True)

# Define rolling periods
period = {'3M': 63, '6M': 126, '12M': 252}
momentum = {}

for label, window in period.items():
    momentum[label] = (1 + daily_port).rolling(window=window).apply(np.prod, raw=True) - 1

#combine all momentums into one df
momentum_df = pd.concat(momentum, axis=1)

#drop any NaN values
momentum_df.dropna(inplace=True)

#add the momentum factors to the original df
my_port = pd.concat([my_port, momentum_df], axis=1)
my_port.head()

Adj Close                                        \
Ticker                             A       AAPL       ABBV ABNB        ABT   
Date                                                                         
2018-01-04 00:00:00+00:00  65.440277  40.705479  73.058868  NaN  52.074825   
2018-01-05 00:00:00+00:00  66.486542  41.168930  74.330681  NaN  52.225338   
2018-01-08 00:00:00+00:00  66.629242  41.016018  73.139755  NaN  52.074825   
2018-01-09 00:00:00+00:00  68.265205  41.011318  73.691093  NaN  52.163368   
2018-01-10 00:00:00+00:00  67.333099  41.001911  73.286766  NaN  52.083683   

                                                                         \
Ticker                          ACGL         ACN        ADBE        ADI   
Date                                                                      
2018-01-04 00:00:00+00:00  28.118069  140.948975  183.220001  80.361885   
2018-01-05 00:00:00+00:00  28.007132  142.111710  185.339996  80.687569   
2018-01-08 00:00:00+00:00  28.010302  143.247360  185.039993  80.828400   
2018-01-09 00:00:00+00:00  27.648960  143.725067  186.699997  80.661163   
2018-01-10 00:00:00+00:00  27.496817  143.220322  187.110001  79.314453   

                                      ... 12M                               \
Ticker                           ADM  ... WTW  WY WYNN XEL XOM XYL YUM ZBH   
Date                                  ...                                    
2018-01-04 00:00:00+00:00  33.174355  ... NaN NaN  NaN NaN NaN NaN NaN NaN   
2018-01-05 00:00:00+00:00  32.952927  ... NaN NaN  NaN NaN NaN NaN NaN NaN   
2018-01-08 00:00:00+00:00  32.879097  ... NaN NaN  NaN NaN NaN NaN NaN NaN   
2018-01-09 00:00:00+00:00  32.985737  ... NaN NaN  NaN NaN NaN NaN NaN NaN   
2018-01-10 00:00:00+00:00  32.731491  ... NaN NaN  NaN NaN NaN NaN NaN NaN   

                                    
Ticker                    ZBRA ZTS  
Date                                
2018-01-04 00:00:00+00:00  NaN NaN  
2018-01-05 00:00:00+00:00  NaN NaN  
2018-01-08 00:00:00+00:00  NaN NaN  
2018-01-09 00:00:00+00:00  NaN NaN  
2018-01-10 00:00:00+00:00  NaN NaN  

[5 rows x 4527 columns]

**Step 4:** Calculate monthly z-factor score for each asset

In [ ]:
# Calculate Z-Factors for each momentum period
my_port['Z-Factor'] = (my_port['3M'] - my_port['3M'].mean()) / my_port['3M'].std()


ValueError: Cannot set a DataFrame with multiple columns to the single column Z-Factor

In [ ]:
z_factors = {}

for label, window in period.items():
  momentum_colum = momentum_df[label]

  monthly_momen = momentum_colum.resample('M').last()
   (window=window).mean()
  std = momentum_colum.rolling(window=window).std()
  z_factors[label] = (momentum_colum - mean)/std

z_factors_df = pd.concat(z_factors, axis=1)


IndentationError: unexpected indent (<ipython-input-8-29ce8d9e69df>, line 7)

In [ ]:
# Initialize a DataFrame for z-scores
z_scores = momentum_df.copy()

# Iterate over the MultiIndex level 0 (e.g., '3M', '6M', '12M')
for label in momentum_df.columns.levels[0]:  # Handle MultiIndex properly
    factor_data = momentum_df[label]
    # Calculate z-scores for each column in this level
    z_scores[label] = (factor_data - factor_data.mean()) / factor_data.std()

# Combine the z-scores into the original DataFrame
my_port = pd.concat([my_port, z_scores], axis=1)

# Display the updated DataFrame
print(my_port.head())


                           Adj Close                                        \
Ticker                             A       AAPL       ABBV ABNB        ABT   
Date                                                                         
2018-01-04 00:00:00+00:00  65.440277  40.705479  73.058868  NaN  52.074825   
2018-01-05 00:00:00+00:00  66.486542  41.168930  74.330681  NaN  52.225338   
2018-01-08 00:00:00+00:00  66.629242  41.016018  73.139755  NaN  52.074825   
2018-01-09 00:00:00+00:00  68.265205  41.011318  73.691093  NaN  52.163368   
2018-01-10 00:00:00+00:00  67.333099  41.001911  73.286766  NaN  52.083683   

                                                                         \
Ticker                          ACGL         ACN        ADBE        ADI   
Date                                                                      
2018-01-04 00:00:00+00:00  28.118069  140.948975  183.220001  80.361885   
2018-01-05 00:00:00+00:00  28.007132  142.111710  185.339996  80.687569   
